Importing Packages and Establish Authentication for eICU Database

In [ ]:
from itertools import combinations
from itertools import permutations
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pickle
from sklearn.metrics import mean_squared_error,r2_score
import matplotlib.colors as colors
from scipy import stats
from datetime import datetime
import time
import warnings
import seaborn as sns
import pickle as pkl
from sklearn.impute import KNNImputer
from IPython.display import display, HTML, Math, Latex
from google.cloud import bigquery
from google.colab import files, auth
import itertools
from functools import reduce
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
name = '2h-pred-final'
import datetime as dt
import os
ts_now = dt.datetime.now()
date = str(ts_now.day) + "." + str(ts_now.month)

base_dir = '/content/drive/My Drive/Colab Notebooks/eICU_OGP/' + name + '/'
data_dir = '/content/drive/My Drive/Colab Notebooks/Data_ALL_Extracted/'

if not os.path.exists(os.path.dirname(base_dir)):
       os.makedirs(os.path.dirname(base_dir))
       print(base_dir)

In [ ]:
auth.authenticate_user()
print('Authenticated')
%load_ext google.colab.data_table

# Function to submit query to BigQuery
def q(query,projectid):
    client = bigquery.Client(location="US",project=projectid)
    # Location must match that of the dataset(s) referenced in the query.
    query_job = client.query(query,
    location="US",)  # API re
    return query_job.to_dataframe()

#Rounding (for heatmap categories)
def myround(x, base):
    return int(base * round(float(x)/base))

def convert_to_datetime(df,time_cols):
    for t_col in time_cols:
        df[t_col] = pd.to_datetime(df[t_col])
    
    return(df)

projectid = "hst-953-2019"
client = bigquery.Client(location="US",project=projectid)
print("Client creating using default project: {}".format(client.project))

read_in = True

Authenticated


# Extracting Data from eICU-CRD

Extracting Insulin Events from Medication Table

In [ ]:
projectid = "hst-953-2019"

#This query was taken from the eicu-code github repository
query="""
-- ------------------------------------------------------------------
-- Title: Extraction of insulin events.
-- Description: This scripts extracts all the boluses of insulin from 
--  the eICU-CRD. This query also distinguish the adminsitration routes: 
--  infusion, bolus pushes and bolus inyections (subcutaneous).
-- ------------------------------------------------------------------

-- Extract the key codes associated to insulin
WITH q1 AS(
SELECT *
FROM `physionet-data.eicu_crd.medication`
WHERE
  drughiclseqno IN(2854, -- NOVOLOG
    18084, -- HUMALOG
    768,   -- REGULAR
    11528, -- LISPRO
    20769, -- NOVOLOG
    13633, -- HUMALOG
    35487) -- ASPART
-- not cancelled    
AND drugordercancelled = 'No'
-- only non-zero dosages
AND dosage IS NOT null
-- insulin dosage is in UNITS
AND UPPER(dosage) LIKE '%UNIT%'
-- must have a start time
AND drugstartoffset is not null
),

-- Extract only drug names that contains the following identifiers
q2 AS(SELECT q1.medicationid, q1.patientunitstayid, q1.drugstartoffset, q1.drugivadmixture,
q1.drugname, q1.dosage, q1.routeadmin, q1.frequency, q1.loadingdose, q1.drugstopoffset,
q1.drugorderoffset
FROM q1
WHERE UPPER(q1.drugname) LIKE '%INSULIN%'
  OR UPPER(q1.drugname) LIKE '%HUMALOG%'
  OR UPPER(q1.drugname) LIKE '%NOVOLOG%'
  OR UPPER(q1.drugname) LIKE '%REGULAR%'
  OR UPPER(q1.drugname) LIKE '%LISPRO%'
  OR UPPER(q1.drugname) LIKE '%NOVOLOG%'
  OR UPPER(q1.drugname) LIKE '%ASPART%'),
  WHERE q1.drugstartoffset < 120
q3 AS(SELECT q2.patientunitstayid, q2.medicationid,
-- Identify boluses and infusions
(CASE
  WHEN UPPER(q2.routeadmin) LIKE '%SUB%' THEN 'BOLUS_INYECTION'
  WHEN UPPER(q2.routeadmin) LIKE 'SQ' THEN 'BOLUS_INYECTION'
  WHEN UPPER(q2.routeadmin) LIKE 'SC' THEN 'BOLUS_INYECTION'
  WHEN UPPER(q2.routeadmin) LIKE 'IVPB' THEN 'BOLUS_PUSH'
  WHEN UPPER(q2.routeadmin) LIKE 'IV PUSH' THEN 'BOLUS_PUSH'
  WHEN UPPER(q2.routeadmin) LIKE 'IV' THEN 'INTRAVENOUS'
  WHEN UPPER(q2.routeadmin) LIKE '%INTRA%' THEN 'INTRAVENOUS'
  WHEN UPPER(q2.routeadmin) LIKE '%CENTRAL%' THEN 'INTRAVENOUS'
  WHEN UPPER(q2.routeadmin) LIKE '%PERI%' THEN 'INTRAVENOUS'
 ELSE null END) AS InsulinAdmin,

-- Colum for insulin drug names
UPPER(q2.drugname) AS InsulinName,
-- Column that contains dose of insulin
UPPER(q2.dosage) AS InsulinDose,
-- due to issue in ETL, times of 0 should likely be null
(CASE WHEN q2.drugorderoffset = 0 THEN null ELSE q2.drugorderoffset END) AS drugorderoffset,
(CASE WHEN q2.drugstartoffset = 0 THEN null ELSE q2.drugstartoffset END) AS drugstartoffset,
(CASE WHEN q2.drugstopoffset = 0 THEN null ELSE q2.drugstopoffset END) AS drugstopoffset,
q2.drugivadmixture,
-- Frequency of insulin admnistration
UPPER(q2.frequency) AS InsulinFrequency,
FROM q2)

SELECT * FROM q3
"""

if read_in:
  ins = pd.read_pickle(data_dir + 'eICU_insulin.pkl')
else:
  ins = q(query,projectid)
  ins.to_pickle(data_dir + 'eICU_insulin.pkl')

Extracting All Lab Values

In [ ]:
#Extracting all glucose values
labq = """
select *, 
FROM `physionet-data.eicu_crd.lab`
WHERE (labname = 'glucose' OR labname = 'bedside glucose' )
order by labresultoffset
"""

if read_in:
  glu = pd.read_pickle(data_dir + 'eICU_glucose.pkl')
else:
  glu = q(labq, projectid)
  glu['fingerstick'] = glu['labname'] == 'bedside glucose'

  #Remove the patients which have less than 3 glucose values
  glu = glu.groupby('patientunitstayid').filter(lambda x : len(x)>3)
  glu.to_pickle(data_dir + 'eICU_glucose.pkl')

In [ ]:
all_pts = list(glu['patientunitstayid'].unique())

Extracting Patient Demographics

In [ ]:
#Extracting patient demographic variables
patientq = """
select patientunitstayid, admissionWeight, admissionheight, ethnicity, hospitalid, age, gender, uniquepid, unitVisitNumber
FROM `physionet-data.eicu_crd.patient`
order by patientunitstayid
"""

if read_in:
  patient = pd.read_pickle(data_dir + 'eICU_patient.pkl')
else:
  patient = q(patientq, projectid)
  for race in ['Caucasian', 'African American', 'Hispanic', 'Asian']:
    patient[race] = (patient.ethnicity == race).astype(int)
  patient['age'] = patient.age.replace('> 89', 90)
  patient['BMI'] = patient.admissionWeight.to_numpy()/np.square(patient.admissionheight.to_numpy()/100)
  patient.to_pickle(data_dir + 'eICU_patient.pkl')

In [ ]:
#Extracting covariates diabetes and hepatic failure
apachpredeq = """
select patientunitstayid, diabetes, hepaticFailure
FROM `physionet-data.eicu_crd.apachepredvar`
"""

if read_in:
  apached_df = pd.read_pickle(data_dir + 'eICU_apached.pkl')
else:
  apached_df = q(apachpredeq, projectid)
  apached_df.to_pickle(data_dir + 'eICU_apached.pkl')

In [ ]:
#Extracting GCS score
gcsq = """
SELECT patientunitstayid, avg(gcs) as gcs_avg

FROM `physionet-data.eicu_crd_derived.pivoted_score` 
GROUP BY patientunitstayid
"""

if read_in:
  gcs_df = pd.read_pickle(data_dir + 'eICU_gcs.pkl')
else:
  gcs_df = q(gcsq, projectid)
  gcs_df.to_pickle(data_dir + 'eICU_gcs.pkl')

In [ ]:
#Extracting other chronic conditions from diagnoses
kidneyq = """

WITH tb AS(
SELECT DISTINCT(patientunitstayid),
  
  CASE 
    WHEN UPPER(diagnosisstring) LIKE "%KIDNEY%" and UPPER(diagnosisstring) LIKE "%CHRONIC%" THEN 1 ELSE 0
  END AS ckd,
  
  CASE 
    WHEN UPPER(diagnosisstring) LIKE "%KIDNEY%" and UPPER(diagnosisstring) LIKE "%ACUTE%"  THEN 1 ELSE 0
  END AS aki,
  
CASE 
    WHEN UPPER(diagnosisstring) LIKE "%KIDNEY%" THEN 1 ELSE 0
  END AS kidneyDisease,
  
  CASE 
    WHEN UPPER(diagnosisstring) LIKE "%LIVER%"  THEN 1 ELSE 0
  END AS liverDisease,
  
  CASE 
    WHEN UPPER(diagnosisstring) LIKE "%CONGESTIVE HEART FAILURE%"  THEN 1 ELSE 0
  END AS congestiveHeartFailure,
  
  
  CASE 
    WHEN UPPER(diagnosisstring) LIKE "%HYPERTENSION%"  THEN 1 ELSE 0
  END AS hypertension,
  
  CASE 
    WHEN UPPER(diagnosisstring) LIKE "%PANCREATITIS%"    THEN 1 ELSE 0
  END AS pancreatitis,
  
  CASE 
    WHEN UPPER(diagnosisstring) LIKE "%PANCREA%" AND UPPER(diagnosisstring) NOT LIKE  "%PANCREATITIS%"  THEN 1 ELSE 0
  END AS pancreasDisease,

  CASE 
    WHEN UPPER(diagnosisstring) LIKE "%SEPSIS%" THEN 1 ELSE 0
  END AS sepsis

  
  
FROM `physionet-data.eicu_crd.diagnosis`
order by patientunitstayid)

SELECT patientunitstayid, MAX(ckd) as ckd, MAX(aki) as aki, MAX(kidneyDisease) as kidneyDisease, MAX(liverDisease) as liverDisease, MAX(hypertension) as hypertension, MAX(congestiveHeartFailure) as congestiveHeartFailure, MAX(pancreasDisease) as pancreasDisease, MAX(pancreatitis) as pancreatitis, MAX(sepsis) as sepsis

FROM tb
group by patientunitstayid
order by patientunitstayid
"""

if read_in:
  kidney_df = pd.read_pickle(data_dir + 'kidney.pkl')

else:
  kidney_df = q(kidneyq,projectid)
  kidney_df.to_pickle(data_dir + 'kidney.pkl')

In [ ]:
#Extracting drugs from medication table
drugsq = """
WITH tb AS(
SELECT DISTINCT(patientunitstayid),
  
  CASE 
    WHEN UPPER(drugname) LIKE '%METFORMIN%' AND drugstartoffset < 1440 THEN 1 ELSE 0
  END AS metformin,
  
  CASE 
    WHEN UPPER(drugname) LIKE '%MORPHINE%' OR UPPER(drugname) LIKE'%HYDROMORPHONE%' AND drugstartoffset < 1440 THEN 1 ELSE 0
  END AS morphine,
  
  CASE 
    WHEN UPPER(drugname) LIKE '%PREDNISOLONE%' AND drugstartoffset < 1440 THEN 1 ELSE 0
  END AS prednisolone,
  
  CASE 
    WHEN UPPER(drugname) LIKE '%DEXA%' AND drugstartoffset < 1440 THEN 1 ELSE 0
  END AS dexamethasone,
  
  CASE 
    WHEN UPPER(drugname) LIKE '%METOPROLOL%' AND drugstartoffset < 1440 THEN 1 ELSE 0
  END AS metoprolol,
  
  #Changed from just being dextrose
  CASE 
    WHEN drugname LIKE '%DEXTROSE 50%-WATER%' 
    OR drugname LIKE '%DEXTROSE 50 % IV SOLN%' 
    OR drugname LIKE '%dextrose 50 % inj%' 
    OR drugname LIKE '%DEXTROSE 50% VIAL%' 
    
    OR UPPER(drugname) LIKE '%DEXTROSE 50%%' 
    OR drugname LIKE '%DEXTROSE (DIABETIC USE) 40 %%' 
    
    AND drugstartoffset > -120 AND drugstartoffset < 120 THEN 1 ELSE 0
  END AS dextrose,
  
FROM `physionet-data.eicu_crd.medication`
order by patientunitstayid)

SELECT patientunitstayid, MAX(metformin) as metformin, MAX(morphine) as morphine, MAX(prednisolone) as prednisolone, MAX(dexamethasone) as dexamethasone, MAX(metoprolol) as metoprolol, MAX(dextrose) as dextrose

FROM tb
group by patientunitstayid
order by patientunitstayid
"""

if read_in:
  drugs_df = pd.read_pickle(data_dir + 'drugs_addl.pkl')
else:
  drugs_df = q(drugsq,projectid)
  drugs_df.to_pickle(data_dir + 'drugs_addl.pkl')

In [ ]:
#Vasopressors extraction
vasoq = """
SELECT patientunitstayid, MAX(norepinephrine) as norepinephrine,  MAX(epinephrine) as epinephrine,	MAX(dopamine)	as dopamine,MAX(phenylephrine)	as phenylephrine,	MAX(vasopressin)	as vasopressin,
FROM `physionet-data.eicu_crd_derived.pivoted_med`
WHERE drugstartoffset < 120
group by patientunitstayid
"""

if read_in:
  vaso = pd.read_pickle(data_dir + 'vaso.pkl')
else:
  vaso = q(vasoq,projectid)
  vaso.to_pickle(data_dir + 'vaso.pkl')

In [ ]:
#Merging all predictor variables
covars_df = reduce(lambda x, y: pd.merge(x, y, on = 'patientunitstayid', how = 'outer'), [patient, apache_df, gcs_df, apached_df, d3_df, dka, drugs_df, kidney_df, vaso])
covars_df = covars_df.drop(['actualicumortality', 'actualiculos', 'ethnicity', 'unitVisitNumber', 'hepaticFailure'], axis = 1)
covars_df = covars_df.drop_duplicates()
covars_df.to_pickle(base_dir + 'covars_df.pkl')
covars_df

Extracting eICU Labs and Vitals:

In [ ]:
#All labs extraction
labquery = """
SELECT p.uniquepid, p.patienthealthsystemstayid, p.patientunitstayid, le.labname, le.labresultoffset

  -- add in some sanity checks on the values; same checks from original MIMIC version
  -- the where clause below requires all labresult to be > 0, so these are only upper limit checks
  , CASE
     WHEN labname = 'albumin' and le.labresult >    10 THEN null -- g/dL 'ALBUMIN'
     WHEN labname = 'anion gap' and le.labresult > 10000 THEN null -- mEq/L 'ANION GAP'
     WHEN labname = '-bands' and le.labresult <     0 THEN null -- immature band forms, %
     WHEN labname = '-bands' and le.labresult >   100 THEN null -- immature band forms, %
     WHEN labname = 'bicarbonate' and le.labresult > 10000 THEN null -- mEq/L 'BICARBONATE'
     WHEN labname = 'HCO3' and le.labresult > 10000 THEN null -- mEq/L 'BICARBONATE'
     WHEN labname = 'bilirubin' and le.labresult >   150 THEN null -- mg/dL 'BILIRUBIN'
     WHEN labname = 'chloride' and le.labresult > 10000 THEN null -- mEq/L 'CHLORIDE'
     WHEN labname = 'creatinine' and le.labresult >   150 THEN null -- mg/dL 'CREATININE'
     WHEN labname = 'glucose' and le.labresult > 10000 THEN null -- mg/dL 'GLUCOSE'
     WHEN labname = 'Hct' and le.labresult >   100 THEN null -- % 'HEMATOCRIT'
     WHEN labname = 'Hgb' and le.labresult >    50 THEN null -- g/dL 'HEMOGLOBIN'
     WHEN labname = 'lactate' and le.labresult >    50 THEN null -- mmol/L 'LACTATE'
     WHEN labname = 'platelets x 1000' and le.labresult > 10000 THEN null -- K/uL 'PLATELET'
     WHEN labname = 'potassium' and le.labresult >    30 THEN null -- mEq/L 'POTASSIUM'
     WHEN labname = 'PTT' and le.labresult >   150 THEN null -- sec 'PTT'
     WHEN labname = 'PT - INR' and le.labresult >    50 THEN null -- 'INR'
     WHEN labname = 'PT' and le.labresult >   150 THEN null -- sec 'PT'
     WHEN labname = 'sodium' and le.labresult >   200 THEN null -- mEq/L == mmol/L 'SODIUM'
     WHEN labname = 'BUN' and le.labresult >   300 THEN null -- 'BUN'
     WHEN labname = 'WBC x 1000' and le.labresult >  1000 THEN null -- 'WBC'
     WHEN labname = 'pH' and le.labresult < 0 THEN null -- 'pH'
     WHEN labname = 'ALT (SGPT)' and le.labresult < 0 THEN null -- 'WBC'
     WHEN labname = 'AST (SGOT)' and le.labresult < 0  THEN null -- 'WBC'
     WHEN labname = 'CRP' and le.labresult < 0  THEN null -- 'WBC'
   ELSE le.labresult
   END AS labresult

  FROM `physionet-data.eicu_crd.patient` p 

  LEFT JOIN `physionet-data.eicu_crd.lab` le
    ON p.patientunitstayid = le.patientunitstayid
    AND le.labname in
    (
    	'anion gap',
    	'albumin',
    	'-bands',
    	'bicarbonate',
    	'HCO3',
    	'total bilirubin',
    	'creatinine',
    	'chloride',
    	'glucose',
    	'Hct',
    	'Hgb',
    	'lactate',
    	'platelets x 1000',
    	'potassium',
    	'PTT',
    	'PT - INR',
    	'PT',
    	'sodium',
    	'BUN',
    	'WBC x 1000',
      'pH',
      'ALT (SGPT)',
      'AST (SGOT)',
      'CRP',
      'triglycerides',
      'total cholestrol',
      'WBC x 1000',
      'RDW',
      'total protein',
      'folate',
      'LDH',
      'ferritin'
    )
    AND labresult IS NOT null AND labresult > 0 -- lab values cannot be 0 and cannot be negative
"""

if read_in:
  labdata = pd.read_pickle(data_dir + 'eICU_labdata.pkl')
else:
  labdata = q(labquery, projectid)
  labdata.to_pickle(data_dir + 'eICU_labdata.pkl')

In [ ]:
#Vitals extraction
vitalquery = """
(with nc as
  (
    select
    patientunitstayid
    , nursingchartoffset
    , nursingchartentryoffset
    , case
    when nursingchartcelltypevallabel = 'Heart Rate'
    and nursingchartcelltypevalname = 'Heart Rate'
    and (REGEXP_CONTAINS(nursingchartvalue, '^[-]?[0-9]+[.]?[0-9]*$'))
    and nursingchartvalue not in ('-','.')
    then cast(nursingchartvalue as FLOAT64	)
    else null end
    as heartrate
    , case
    when nursingchartcelltypevallabel = 'Respiratory Rate'
    and nursingchartcelltypevalname = 'Respiratory Rate'
    and (REGEXP_CONTAINS(nursingchartvalue, '^[-]?[0-9]+[.]?[0-9]*$'))
    and nursingchartvalue not in ('-','.')
    then cast(nursingchartvalue as FLOAT64	)
    else null end
    as RespiratoryRate
    , case
    when nursingchartcelltypevallabel = 'O2 Saturation'
    and nursingchartcelltypevalname = 'O2 Saturation'
    and (REGEXP_CONTAINS(nursingchartvalue, '^[-]?[0-9]+[.]?[0-9]*$'))
    and nursingchartvalue not in ('-','.')
    then cast(nursingchartvalue as FLOAT64	)
    else null end
    as o2saturation
    , case
    when nursingchartcelltypevallabel = 'Non-Invasive BP'
    and nursingchartcelltypevalname = 'Non-Invasive BP Systolic'
    and (REGEXP_CONTAINS(nursingchartvalue, '^[-]?[0-9]+[.]?[0-9]*$'))
    and nursingchartvalue not in ('-','.')
    then cast(nursingchartvalue as FLOAT64	)
    else null end
    as nibp_systolic
    , case
    when nursingchartcelltypevallabel = 'Non-Invasive BP'
    and nursingchartcelltypevalname = 'Non-Invasive BP Diastolic'
    and (REGEXP_CONTAINS(nursingchartvalue, '^[-]?[0-9]+[.]?[0-9]*$'))
    and nursingchartvalue not in ('-','.')
    then cast(nursingchartvalue as FLOAT64	)
    else null end
    as nibp_diastolic
    , case
    when nursingchartcelltypevallabel = 'Non-Invasive BP'
    and nursingchartcelltypevalname = 'Non-Invasive BP Mean'
    and (REGEXP_CONTAINS(nursingchartvalue, '^[-]?[0-9]+[.]?[0-9]*$'))
    and nursingchartvalue not in ('-','.')
    then cast(nursingchartvalue as FLOAT64	)
    else null end
    as nibp_mean
    , case
    when nursingchartcelltypevallabel = 'Temperature'
    and nursingchartcelltypevalname = 'Temperature (C)'
    and (REGEXP_CONTAINS(nursingchartvalue, '^[-]?[0-9]+[.]?[0-9]*$'))
    and nursingchartvalue not in ('-','.')
    then cast(nursingchartvalue as FLOAT64	)
    else null end
    as temperature
    , case
    when nursingchartcelltypevallabel = 'Temperature'
    and nursingchartcelltypevalname = 'Temperature Location'
    then nursingchartvalue
    else null end
    as TemperatureLocation
    , case
    when nursingchartcelltypevallabel = 'Invasive BP'
    and nursingchartcelltypevalname = 'Invasive BP Systolic'
    and (REGEXP_CONTAINS(nursingchartvalue, '^[-]?[0-9]+[.]?[0-9]*$'))
    and nursingchartvalue not in ('-','.')
    then cast(nursingchartvalue as FLOAT64	)
    else null end
    as ibp_systolic
    , case
    when nursingchartcelltypevallabel = 'Invasive BP'
    and nursingchartcelltypevalname = 'Invasive BP Diastolic'
    and (REGEXP_CONTAINS(nursingchartvalue, '^[-]?[0-9]+[.]?[0-9]*$'))
    and nursingchartvalue not in ('-','.')
    then cast(nursingchartvalue as FLOAT64	)
    else null end
    as ibp_diastolic
    , case
    when nursingchartcelltypevallabel = 'Invasive BP'
    and nursingchartcelltypevalname = 'Invasive BP Mean'
    and (REGEXP_CONTAINS(nursingchartvalue, '^[-]?[0-9]+[.]?[0-9]*$'))
    and nursingchartvalue not in ('-','.')
    then cast(nursingchartvalue as FLOAT64	)
    -- other map fields
    when nursingchartcelltypevallabel = 'MAP (mmHg)'
    and nursingchartcelltypevalname = 'Value'
    and (REGEXP_CONTAINS(nursingchartvalue, '^[-]?[0-9]+[.]?[0-9]*$'))
    and nursingchartvalue not in ('-','.')
    then cast(nursingchartvalue as FLOAT64	)
    when nursingchartcelltypevallabel = 'Arterial Line MAP (mmHg)'
    and nursingchartcelltypevalname = 'Value'
    and (REGEXP_CONTAINS(nursingchartvalue, '^[-]?[0-9]+[.]?[0-9]*$'))
    and nursingchartvalue not in ('-','.')
    then cast(nursingchartvalue as FLOAT64	)
    else null end
    as ibp_mean
    from `physionet-data.eicu_crd.nursecharting` nursecharting
    -- speed up by only looking at a subset of charted data
    where nursingchartcelltypecat in
    (
      'Vital Signs','Scores','Other Vital Signs and Infusions'
    )
  )
  select
  patientunitstayid
  , nursingchartoffset as chartoffset
  , nursingchartentryoffset as entryoffset
  , avg(case when heartrate >= 25 and heartrate <= 225 then heartrate else null end) as heartrate
  , avg(case when RespiratoryRate >= 0 and RespiratoryRate <= 60 then RespiratoryRate else null end) as RespiratoryRate
  , avg(case when o2saturation >= 0 and o2saturation <= 100 then o2saturation else null end) as spo2
  , avg(case when nibp_systolic >= 25 and nibp_systolic <= 250 then nibp_systolic else null end) as nibp_systolic
  , avg(case when nibp_diastolic >= 1 and nibp_diastolic <= 200 then nibp_diastolic else null end) as nibp_diastolic
  , avg(case when nibp_mean >= 1 and nibp_mean <= 250 then nibp_mean else null end) as nibp_mean
  , avg(case when temperature >= 25 and temperature <= 46 then temperature else null end) as temperature
  , max(temperaturelocation) as temperaturelocation
  , avg(case when ibp_systolic >= 1 and ibp_systolic <= 300 then ibp_systolic else null end) as ibp_systolic
  , avg(case when ibp_diastolic >= 1 and ibp_diastolic <= 200 then ibp_diastolic else null end) as ibp_diastolic
  , avg(case when ibp_mean >= 1 and ibp_mean <= 250 then ibp_mean else null end) as ibp_mean
  from nc
  WHERE heartrate IS NOT NULL
  OR RespiratoryRate IS NOT NULL
  OR o2saturation IS NOT NULL
  OR nibp_systolic IS NOT NULL
  OR nibp_diastolic IS NOT NULL
  OR nibp_mean IS NOT NULL
  OR temperature IS NOT NULL
  OR temperaturelocation IS NOT NULL
  OR ibp_systolic IS NOT NULL
  OR ibp_diastolic IS NOT NULL
  OR ibp_mean IS NOT NULL
  group by patientunitstayid, nursingchartoffset, nursingchartentryoffset
  order by patientunitstayid, nursingchartoffset, nursingchartentryoffset)
"""

if read_in:
  vitaldata = pd.read_pickle(data_dir + 'eICU_vitaldata.pkl')
else:
  vitaldata = q(vitalquery,projectid)
  vitaldata.to_pickle(data_dir + 'eICU_vitaldata.pkl')


# Processing Data

In [ ]:
final_pts = glu.patientunitstayid.unique()

In [ ]:
#Predictor variables all collected within first two hours
ob_end = 2*60
gap = 0 * 60
pred_start = ob_end + gap

In [ ]:
#Processing Labs and vitals
labs_df = labdata[labdata['labresultoffset'] < ob_end].groupby(['patientunitstayid', 'labname']).mean()
labs_df.reset_index(inplace=True)  
labs_df = labs_df.pivot(index = 'patientunitstayid', columns = 'labname', values = 'labresult')

vital_df = vitaldata[vitaldata['chartoffset'] < ob_end].groupby(['patientunitstayid']).mean().drop(['chartoffset', 'entryoffset'], axis = 1)

In [ ]:
#Merging values
labs_vitals_df = pd.merge(labs_df, vital_df, on = 'patientunitstayid', how = 'inner')
labs_vitals_glu = pd.merge(glu_hypo, labs_vitals_df, on = 'patientunitstayid', how = 'outer')

#Identifying patients who had hypoglycemia in a previous patient visit
multistay_pts = patient[patient['unitVisitNumber'] > 1].patientunitstayid.values
min_glu = glu[['labresult', 'patientunitstayid']].groupby(by = 'patientunitstayid').min()
multistay_pts = list(set(multistay_pts) & set(min_glu.index.values))
prev_hypo = min_glu.loc[multistay_pts]['labresult'] < 72
min_glu['prev_hypo'] = prev_hypo

#Computing BGCV in first two hours and merging
cv = lambda x: np.std(x, ddof=1) / np.mean(x) * 100 
obs_glu = glu[glu['labresultoffset'] < ob_end]
cv_df = obs_glu[['labresult', 'patientunitstayid']].groupby(by = 'patientunitstayid').apply(cv).drop('patientunitstayid', axis = 1).rename(columns = {'labresult': 'cv_glucose'})
l_v_glu_prevvist = pd.merge(min_glu[['prev_hypo', 'patientunitstayid']], labs_vitals_glu, on = 'patientunitstayid', how = 'outer')
event_df = pd.merge(covars_df, l_v_glu_prevvist, on = 'patientunitstayid', how = 'inner')
event_df = pd.merge(cv_df, event_df, on = 'patientunitstayid', how = 'right')

In [ ]:
#Hypoglycemic outcome, happening after prediction horizon
outcome_glu = glu[glu['labresultoffset'] > pred_start]
event_df = pd.merge(event_df, outcome_glu[['labresult', 'patientunitstayid']].groupby(by = 'patientunitstayid').min(), on = 'patientunitstayid', how = 'inner')
event_df = pd.merge(event_df, insulin_df, on = 'patientunitstayid', how = 'outer')

In [ ]:
#Dropping patients with more than 10 missing variables and performign KNN imputation
event_df = event_df.dropna(axis = 0, thresh = 20)
event_df = KNNImputer(event_df)

In [ ]:
#Saving files
event_df.to_pickle(base_dir + 'event_df.pkl')
event_df.to_csv(base_dir + 'event_df_GCP.csv')